In [ ]:
%matplotlib notebook

# General libraries
import numpy as np                # to deal with arrays, vectors, matrices...
import matplotlib.pyplot as plt   # to plot the data
import matplotlib.gridspec as gridspec

# Tensorflow
import os
HOME = os.getenv('HOME')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # to get rid of the TF compilation warnings
import tensorflow as tf
from tensorflow.keras import models, datasets
from tensorflow.keras.layers import Dense, LSTM, RepeatVector, TimeDistributed

In [ ]:
# Only because my system-wide config is tuned, you don't need these lines
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = 5,3
mpl.rcParams['font.size'] = 6.0

In [ ]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the dataset
        if out_end_ix > len(sequences): break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [ ]:
# define input sequence
in_seq1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = np.array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

In [ ]:
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = np.hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps_in, n_steps_out = 4, 2
# covert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)

In [ ]:
# define model
model = models.Sequential()
model.add(LSTM(200, activation='relu', input_shape=X.shape[1:]))
model.add(RepeatVector(n_steps_out))
model.add(LSTM(200, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(X.shape[-1],activation='relu')))

model.summary()

In [ ]:
model.compile(optimizer='adam', loss='mse')

In [ ]:
# fit model
history = model.fit(X, y, epochs=200, verbose=1)

In [ ]:
err = history.history['loss']
# val_err = history.history['val_loss']
# acc = history.history['accuracy']
fig, ax = plt.subplots()
ax.plot(err,label='loss')
# ax.plot(val_err,label='Val Loss')
# ax.plot(acc,label='accuracy')
ax.set_title('Learning curve')
plt.show()

In [ ]:
# demonstrate prediction
x_input = np.array([[60, 65, 125], [70, 75, 145], [80, 85, 165], [90, 95, 185]])
x_input = x_input.reshape((1, n_steps_in, X.shape[-1]))
print('Input:')
print(x_input)
expected = np.array([[100,105,205],[110,115,225]])
yhat = model.predict(x_input, verbose=0)
print('\nExpected:')
print(expected)
print('predict:')
print(np.round(yhat,1))